###**Importing Libraries**

In [1]:
#!pip install langdetect
#!pip install googletrans==4.0.0-rc1

In [2]:
import pandas as pd
import numpy as np
import scikitplot as skplt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

###**Import your dataset**

In [3]:
from googletrans import Translator
translator = Translator()


In [4]:
#reading the .xlsx dataset in to the dataframe
df = pd.read_csv("AIDA-Dataset_csv.csv", encoding = "utf-8")

In [5]:
df.shape

(520, 107)

###**Data preprossesing**

In [6]:
short_names_mapping = {
    'Do you want to participate in this survey?': 'Participation',
    'Please state your age? (e.g. 25)': 'Age',
    'What is your gender?': 'Gender',
    'What degree are you currently pursuing?': 'Degree',
    'What is your current academic year? (e.g. 2)': 'AcademicYear',
    'Which medium of school have you come from?': 'SchoolMedium',
    'Which department do you study in?': 'Department',
    'What is your current cumulative grade points average (CGPA)?': 'CGPA',
    'Do you have any personal history of depression?': 'DepressionHistory',
    'What is your current relationship status?': 'RelationshipStatus',
    'What is your religion?': 'Religion',
    'What is your current place of residence?': 'Residence',
    'Currently I live - ': 'CurrentLocation',
    'Number of siblings (e.g. 2)': 'SiblingsCount',
    'What is your ordinal position/birth order among your siblings? (e.g. 1)': 'SiblingOrder',
    ' [I feel that I have additional responsibility to be in this position.]': 'ResponsibilityFeeling',
    'What is your father’s occupation? (If you do not see the occupation listed below or in case of a deceased father please choose the "other" option.)': 'FatherOccupation',
    'What is your mother’s occupation? (If you do not see the occupation listed below or in case of a deceased mother please choose the "other" option.)': 'MotherOccupation',
    'What is your family’s (household) average monthly income? (e.g. 2000)': 'FamilyIncome',
    'Do you have any financial hardship currently?': 'FinancialHardship',
    'Is your family financially dependent on you?': 'FinancialDependency',
    'Do you have anyone in your family who has been suffering from fatal diseases?': 'FamilyIllness',
    ' [How often do you communicate with your parents?]': 'ParentalCommunicationFrequency',
    'Give your opinion about these: [My parents are very strict about my daily life style]': 'ParentalStrictness',
    'Give your opinion about these: [My parents give pressure for my academic subject selection]': 'ParentalAcademicPressure',
    'Give your opinion about these: [My parents give pressure for my marriage.]': 'ParentalMarriagePressure',
    'Give your opinion about these: [My parents give pressure for my career choice.]': 'ParentalCareerPressure',
    'Give your opinion about these: [My parents give pressure regarding my decision on studying abroad]': 'ParentalStudyAbroadPressure',
    'Give your opinion about these statements [I have bad understanding with my parents]': 'ParentalUnderstanding',
    'Give your opinion about these statements [I have bad bonding with my siblings]': 'SiblingBonding',
    'Give your opinion about these statements [My  parents have unstable relationship]': 'ParentalRelationshipStability',
    'Give your opinion about these statements [I don’t get along with most of my classmates/friends]': 'PeerRelationship',
    'Give your opinion about these statements [My university teachers are unsupportive]': 'TeacherSupport',
    'Give your opinion about these statements [An unstable relationship with my partner affects my life.]': 'PartnerRelationshipImpact',
    'Give your opinion about these statements [A discouraging working environment (colleague/boss) affects the output of my work.]': 'WorkEnvironmentImpact',
    ' [I feel society interferes with my freedom of expression (freedom of speech / movement / etc...)]': 'SocietalInterference',
    'Have you ever experienced this sort of violence before? [I have faced Physical Violence before. (someone uses his/her body part or an object to control my actions) (শারিরিকনির্যাতন)]': 'PhysicalViolenceExperience',
    'Have you ever experienced this sort of violence before? [I have faced Sexual Violence before. (i am/was forced to unwillingly take part in sexual activity) (যৌনসহিংসতা)]': 'SexualViolenceExperience',
    'Have you ever experienced this sort of violence before? [I have faced Verbal Violence before. (someone uses language, whether spoken or written as an abusing tool against me) (মৌখিকগালি)]': 'VerbalViolenceExperience',
    'Have you ever experienced this sort of violence before? [I have faced Emotional Violence before. (someone says or does something to make me feel stupid or worthless) (মানসিকহিংসা)]': 'EmotionalViolenceExperience',
    'What kind of institution do you study in?': 'InstitutionType',
    'Self-perception on Academic status: [My academic performance is very poor.]': 'PoorAcademicPerformanceSelfPerception',
    'Self-perception on Academic status: [I have been criticized for my academic result.]': 'AcademicCriticismSelfPerception',
    'Self-perception on Academic status: [I am unsatisfied with my academic workload (i.e., presentations, assignments, tutorials)]': 'UnsatisfiedAcademicWorkloadSelfPerception',
    'Give your opinion for these: [I am not studying in my subject of interest.]': 'NonInterestSubjectOpinion',
    'Give your opinion for these: [I am unhappy with the subject that I am studying.]': 'UnhappySubjectOpinion',
    'Give your opinion for these: [I am not studying in the institution of my interest.]': 'NonInterestInstitutionOpinion',
    'Give your opinion for these: [I am unhappy with the institution that I am studying.]': 'UnhappyInstitutionOpinion',
    'Did you select the subject due to career prospects only?': 'CareerProspectsSubjectSelection',
    'What is your current employment status?': 'EmploymentStatus',
    'Which of the following is true for you?': 'PersonalSituation',
    'How the pandemic is affecting your full time /part time job/tutoring?': 'PandemicImpactOnWork',
    'I am satisfied with my current job.': 'JobSatisfaction',
    'My family/partner is satisfied with my current job.': 'FamilyPartnerJobSatisfaction',
    ' [In my opinion, enough jobs are not available related to my pursued subject. (your academic subject )]': 'InsufficientJobsSubjectOpinion',
    'How many close friends (approximately) do you have? (e.g. 3)': 'CloseFriendsCount',
    ' [How often do you hang out with your friends?]': 'FriendHangoutFrequency',
    'Are you involved in any extracurricular activities?': 'ExtracurricularInvolvement',
    'How strongly do you agree with each of the following questions: [How often do you smoke?]': 'SmokingFrequencyPerception',
    'How strongly do you agree with each of the following questions: [How often do you take drugs?]': 'DrugUseFrequencyPerception',
    'How strongly do you agree with each of the following questions: [How often do you drink alcohol?]': 'AlcoholConsumptionFrequencyPerception',
    'How strongly do you agree with each of the following questions: [How often do you engage in physical exercise?]': 'PhysicalExerciseFrequencyPerception',
    'How strongly do you agree with each of the following questions: [How often do you practice religious activities?]': 'ReligiousActivitiesFrequencyPerception',
    'How strongly do you agree with each of the following questions: [How often do you play video games?]': 'VideoGamePlayingFrequencyPerception',
    'How strongly do you agree with each of the following questions: [How often do you play sports?]': 'SportsPlayingFrequencyPerception',
    'On average how many hours do you spend in social media? (e.g. 5)': 'SocialMediaUsageHours',
    'Give your opinion on these: [The success stories of my peers in social media sometimes pose negative impact on me]': 'PeerSuccessImpactOnSocialMediaOpinion',
    'Give your opinion on these: [Social media has negative influence on my purchase decision]': 'SocialMediaInfluenceOnPurchaseDecisionOpinion',
    "Give your opinion on these: [I find human bonding less trust-worthy because it's easy to become imposters on social media]": 'TrustInHumanBondingOnSocialMediaOpinion',
    'Give your opinion on these: [I feel numbness of feelings as I am constantly seeing variety of incidents on social media.]': 'NumbnessFeelingsDueToSocialMediaOpinion',
    'Give your opinion on these: [I like to interact with my friends in person rather than in virtual mode.]': 'PreferenceForInPersonInteractionOpinion',
    'Sad': 'Sadness',
    'Depressed mood (sadness, hopeless, helpless, worthless)': 'DepressedMood',
    'Future': 'FutureOutlook',
    'Failure': 'FearOfFailure',
    'Satisfaction': 'SatisfactionLevel',
    'Guilty ': 'FeelingsOfGuilt',
    'Feelings of guiltiness ': 'GuiltFeelings',
    'Punished': 'FeelingsOfPunishment',
    'Disappointment ': 'FeelingsOfDisappointment',
    'Blame on me': 'SelfBlame',
    'Cry ': 'TendencyToCry',
    'Irritated': 'Irritability',
    'Interest in other people': 'InterestInOthers',
    'Decisions': 'DecisionMaking',
    'Look': 'AppearanceConcern',
    'Self-Care (This question is pertaining to what you feeling right now)': 'SelfCarePerception',
    'Energy Level:': 'EnergyLevel',
    'Work and Activities': 'WorkAndActivitiesLevel',
    'Sleep ': 'SleepQuality',
    'Sleeping Too Much:': 'ExcessiveSleeping',
    'Tired': 'Fatigue',
    'Appetite': 'AppetiteLevel',
    'a)\tAccording to you:': 'SubjectiveHealthPerception',
    'b)\tAccording to weekly measurements:': 'ObjectiveHealthPerception',
    'Feelings of suicide': 'SuicidalFeelings',
    'Retardation (প্রতিবন্ধকতা) (slowness of thought and speech, impaired ability to concentrate, decreased motor activity)/ (চিন্তা , কথা , মনোযোগ , মস্তিষ্কের কর্মক্ষমতা হ্রাস)': 'CognitiveRetardation',
    'Agitation (feeling of irritability or severe restlessness) (উত্কণ্ঠা)': 'Agitation',
    'Anxiety Psychic (মানসিক দুশ্চিন্তা)': 'PsychicAnxiety',
    ' [I felt that I was rather touchy/oversensitive over the past week]': 'SensitivityPerception',
    ' [I was aware of the action of my heart in the absence of physical exertion (e.g. sense of heart rate increase, heart missing a beat)]': 'HeartAwarenessPerception',
    'ANXIETY SOMATIC (দৈহিক) (physiological concomitants of anxiety) such as\ngastro-intestinal – dry mouth, wind, indigestion (বদ হজম), diarrhea, cramps ( পেটে ব্যাথা), belching( বমি)\ncardio-vascular – palpitations ( বুক ধড়ফড়) , headaches\nrespiratory – hyperventilation ( জোরে জোরে শ্বাস নেয়া), sighing urinary frequency sweating': 'SomaticAnxietySymptoms',
    'Somatic Symptoms Gastro-Intestinal': 'GastrointestinalSymptoms',
    'General Somatic Symptoms': 'GeneralSomaticSymptoms',
    'Genital Symptoms (symptoms such as loss of libido, menstrual disturbances)': 'GenitalSymptoms',
    'Hypochondriasis (The belief and fear of serious illness) / ( মরণব্যাধি আছে এমন ভ্রান্ত ধারনা )': 'Hypochondriasis',
    'Insight (self-perception)': 'SelfPerception',
}
df.rename(columns=short_names_mapping, inplace=True)


###**Feature Engineering**

In [7]:
# demographic_feature=['Age',
#  'Gender',
#  'Degree',
#  'SchoolMedium',
#  'CGPA']

In [8]:
# df_demography = df[demographic_feature]
# df_demography.head()

In [9]:
df.columns.tolist()

['Participation',
 'Age',
 'Gender',
 'Degree',
 'AcademicYear',
 'SchoolMedium',
 'Department',
 'CGPA',
 'DepressionHistory',
 'RelationshipStatus',
 'Religion',
 'Residence',
 'CurrentLocation',
 'SiblingsCount',
 'SiblingOrder',
 'ResponsibilityFeeling',
 'FatherOccupation',
 'MotherOccupation',
 'FamilyIncome',
 'FinancialHardship',
 'FinancialDependency',
 'FamilyIllness',
 'ParentalCommunicationFrequency',
 'ParentalStrictness',
 'ParentalAcademicPressure',
 'ParentalMarriagePressure',
 'ParentalCareerPressure',
 'ParentalStudyAbroadPressure',
 'ParentalUnderstanding',
 'SiblingBonding',
 'ParentalRelationshipStability',
 'PeerRelationship',
 'TeacherSupport',
 'PartnerRelationshipImpact',
 'WorkEnvironmentImpact',
 'SocietalInterference',
 'PhysicalViolenceExperience',
 'SexualViolenceExperience',
 'VerbalViolenceExperience',
 'EmotionalViolenceExperience',
 'InstitutionType',
 'PoorAcademicPerformanceSelfPerception',
 'AcademicCriticismSelfPerception',
 'UnsatisfiedAcademicWor

In [10]:
df.head()

,Participation,Age,Gender,Degree,AcademicYear,SchoolMedium,Department,CGPA,DepressionHistory,RelationshipStatus,...,Agitation,PsychicAnxiety,SensitivityPerception,HeartAwarenessPerception,SomaticAnxietySymptoms,GastrointestinalSymptoms,GeneralSomaticSymptoms,GenitalSymptoms,Hypochondriasis,SelfPerception
0,Yes,23,Female,Undergraduate,4,Bangla Medium,Mathematical/Engineering & Physical Sciences,3.38,No,Single,...,Fidgetiness/ (অস্থিরতা),Subjective(depending on patient/subject) tensi...,some of the time,not at all,Absent.,None.,None.,Absent.,Not present.,Denies being ill at all.
1,Yes,24,Female,Postgraduate,1,English Medium,Business & Economics Studies,3.08,Yes,Single,...,None.,Worrying about minor matters. (ছোটখাট বিষয় নিয়...,not at all,not at all,Mild.,None.,None.,Absent.,Not present.,Denies being ill at all.
2,Yes,24,Male,Undergraduate,4,Bangla Medium,Mathematical/Engineering & Physical Sciences,3.05,Yes,Prefer not to say,...,Fidgetiness/ (অস্থিরতা),Subjective(depending on patient/subject) tensi...,some of the time,a good part of time,Absent.,None.,"Heaviness in limbs, back or head. Backaches, h...",Absent.,Not present.,Acknowledges being depressed and ill.
3,Yes,22,Male,Postgraduate,1,Bangla Medium,Mathematical/Engineering & Physical Sciences,3.82,Yes,Single,...,None.,Worrying about minor matters. (ছোটখাট বিষয় নিয়...,a good part of time,some of the time,Mild.,None.,None.,Absent.,Not present.,Denies being ill at all.
4,Yes,25,Male,Undergraduate,4,Bangla Medium,Business & Economics Studies,3.40,Yes,Single,...,"Moving about, can’t sit still.",Subjective(depending on patient/subject) tensi...,some of the time,some of the time,Mild.,Loss of appetite but eating without staff enco...,"Heaviness in limbs, back or head. Backaches, h...",Severe.,Preoccupation (আচ্ছন্ন) with health,Acknowledges being depressed and ill.


In [12]:
df2 = pd.DataFrame()

In [13]:
df2.head()

""


In [14]:

demographics = ['Age', 'Gender', 'Religion', 'Residence']

academic = ['Degree','AcademicYear','SchoolMedium', 'Department', 'CGPA', 'TeacherSupport', 'InstitutionType',
            'PoorAcademicPerformanceSelfPerception','AcademicCriticismSelfPerception',
            'UnsatisfiedAcademicWorkloadSelfPerception','NonInterestSubjectOpinion','UnhappySubjectOpinion',
            'NonInterestInstitutionOpinion','UnhappyInstitutionOpinion',]


lifestyle = ['RelationshipStatus','CurrentLocation',  'CloseFriendsCount',
             'FriendHangoutFrequency','ExtracurricularInvolvement', 'SmokingFrequencyPerception',
 'DrugUseFrequencyPerception','AlcoholConsumptionFrequencyPerception','PhysicalExerciseFrequencyPerception',
             'ReligiousActivitiesFrequencyPerception','VideoGamePlayingFrequencyPerception',
             'SportsPlayingFrequencyPerception','SocialMediaUsageHours','PeerSuccessImpactOnSocialMediaOpinion',
 'SocialMediaInfluenceOnPurchaseDecisionOpinion','TrustInHumanBondingOnSocialMediaOpinion',]


work = [ 'CareerProspectsSubjectSelection', 'EmploymentStatus', 'PersonalSituation', 'PandemicImpactOnWork',
 'JobSatisfaction','FamilyPartnerJobSatisfaction','InsufficientJobsSubjectOpinion']

health = ['DepressionHistory',  'Sadness','DepressedMood',  'SensitivityPerception',
 'HeartAwarenessPerception','SomaticAnxietySymptoms','GastrointestinalSymptoms','GeneralSomaticSymptoms',
 'GenitalSymptoms','Hypochondriasis']

energy = [ 'EnergyLevel','WorkAndActivitiesLevel','SleepQuality','ExcessiveSleeping','Fatigue',
 'AppetiteLevel','SubjectiveHealthPerception','ObjectiveHealthPerception', 'Agitation']

emotions = ['FutureOutlook','FearOfFailure','SatisfactionLevel','FeelingsOfGuilt','GuiltFeelings',
 'FeelingsOfPunishment','FeelingsOfDisappointment','SelfBlame','TendencyToCry','Irritability',
 'InterestInOthers','DecisionMaking','AppearanceConcern','SelfCarePerception', 'SuicidalFeelings',
 'CognitiveRetardation','PsychicAnxiety', 'SelfPerception', 'ResponsibilityFeeling']

family = ['SiblingsCount','SiblingOrder',  'FatherOccupation', 'MotherOccupation', 'FamilyIncome', 'FinancialHardship',
          'FinancialDependency', 'FamilyIllness', 'ParentalCommunicationFrequency', 'ParentalStrictness', 
         'ParentalAcademicPressure', 'ParentalMarriagePressure', 'ParentalCareerPressure', 
          'ParentalStudyAbroadPressure', 'ParentalUnderstanding', 'SiblingBonding','ParentalRelationshipStability'
         ,'PartnerRelationshipImpact', ]

social = ['PeerRelationship', 'WorkEnvironmentImpact', 'SocietalInterference',  'NumbnessFeelingsDueToSocialMediaOpinion',
 'PreferenceForInPersonInteractionOpinion']

violence = ['PhysicalViolenceExperience', 'SexualViolenceExperience', 'VerbalViolenceExperience', 'EmotionalViolenceExperience']

# Covered Features:

    Demographics: Age, Gender, Religion, Residence
    Academic: Degree, AcademicYear, SchoolMedium, Department, CGPA, TeacherSupport, InstitutionType, PoorAcademicPerformanceSelfPerception, AcademicCriticismSelfPerception, UnsatisfiedAcademicWorkloadSelfPerception
    Lifestyle: RelationshipStatus, Religion, Residence, CurrentLocation, ExtracurricularInvolvement, SmokingFrequencyPerception, DrugUseFrequencyPerception, AlcoholConsumptionFrequencyPerception, PhysicalExerciseFrequencyPerception, ReligiousActivitiesFrequencyPerception, VideoGamePlayingFrequencyPerception, SportsPlayingFrequencyPerception, SocialMediaUsageHours, PeerSuccessImpactOnSocialMediaOpinion, SocialMediaInfluenceOnPurchaseDecisionOpinion
    Work: CareerProspectsSubjectSelection, EmploymentStatus, PandemicImpactOnWork, JobSatisfaction, FamilyPartnerJobSatisfaction, InsufficientJobsSubjectOpinion
    Health: DepressionHistory, Sadness, DepressedMood, SomaticAnxietySymptoms, GastrointestinalSymptoms, GeneralSomaticSymptoms
    Energy: EnergyLevel, WorkAndActivitiesLevel, SleepQuality, ExcessiveSleeping, Fatigue, AppetiteLevel, SubjectiveHealthPerception, ObjectiveHealthPerception
    Emotions: FutureOutlook, FearOfFailure, SatisfactionLevel, FeelingsOfGuilt, FeelingsOfPunishment, FeelingsOfDisappointment, SelfBlame, TendencyToCry, Irritability, InterestInOthers, DecisionMaking, CognitiveRetardation, PsychicAnxiety
    Family: SiblingsCount, SiblingOrder, FatherOccupation, MotherOccupation, FamilyIncome, FinancialHardship, FinancialDependency, FamilyIllness, ParentalCommunicationFrequency, ParentalStrictness, ParentalUnderstanding, SiblingBonding, ParentalRelationshipStability
    Social: PeerRelationship, WorkEnvironmentImpact, SocietalInterference, PreferenceForInPersonInteractionOpinion
    Violence: PhysicalViolenceExperience, SexualViolenceExperience

Features Not Covered (or not specifically mentioned if grouped under broader terms):

    Family: ParentalAcademicPressure, ParentalMarriagePressure, ParentalCareerPressure, ParentalStudyAbroadPressure, PartnerRelationshipImpact
    Health: HeartAwarenessPerception, GenitalSymptoms, Hypochondriasis
    Emotions: AppearanceConcern, SelfCarePerception, SuicidalFeelings, SelfPerception, ResponsibilityFeeling
    Social: NumbnessFeelingsDueToSocialMediaOpinion, TrustInHumanBondingOnSocialMediaOpinion
    Violence: VerbalViolenceExperience, EmotionalViolenceExperience

In [21]:
# Assuming df is your DataFrame
def create_description(row):
    # Demographics
    demographics = f"{row['Gender']} aged {row['Age']}, practices {row['Religion']}, lives in {row['Residence']}."
    
    # Academic details
    academic = f"Studying {row['Degree']} in {row['Department']} during {row['AcademicYear']}, with a CGPA of {row['CGPA']}, medium of instruction is {row['SchoolMedium']}. Supports from teachers: {row['TeacherSupport']}, studying in a {row['InstitutionType']}."

    # Lifestyle choices
    lifestyle = f"{row['RelationshipStatus']}, practicing {row['Religion']} in {row['Residence']}, currently in {row['CurrentLocation']}. Engages in social activities like {row['ExtracurricularInvolvement']}, smoking perception: {row['SmokingFrequencyPerception']}, drug use: {row['DrugUseFrequencyPerception']}, alcohol consumption: {row['AlcoholConsumptionFrequencyPerception']}. Exercises {row['PhysicalExerciseFrequencyPerception']} times, involved in religious activities {row['ReligiousActivitiesFrequencyPerception']}. Plays video games {row['VideoGamePlayingFrequencyPerception']} and sports {row['SportsPlayingFrequencyPerception']}. Uses social media {row['SocialMediaUsageHours']} hours daily, impacted by peers on social media {row['PeerSuccessImpactOnSocialMediaOpinion']}, and social media influences on purchasing decisions {row['SocialMediaInfluenceOnPurchaseDecisionOpinion']}."

    # Work and career
    work = f"Career prospects influenced by {row['CareerProspectsSubjectSelection']}. Status: {row['EmploymentStatus']}, situation affected by the pandemic {row['PandemicImpactOnWork']}. Job satisfaction: {row['JobSatisfaction']}, family or partner's job satisfaction: {row['FamilyPartnerJobSatisfaction']}. Opinion on job availability: {row['InsufficientJobsSubjectOpinion']}."

    # Health and well-being
    health = f"Perceived depression history: {row['DepressionHistory']}, currently feels {row['Sadness']} and {row['DepressedMood']}. Experiences somatic anxiety {row['SomaticAnxietySymptoms']}, gastrointestinal issues {row['GastrointestinalSymptoms']}, and general somatic symptoms {row['GeneralSomaticSymptoms']}."

    # Energy levels and sleep
    energy = f"Energy levels perceived as {row['EnergyLevel']}, engaged in work and activities level {row['WorkAndActivitiesLevel']}. Quality of sleep: {row['SleepQuality']}, tendency to sleep excessively {row['ExcessiveSleeping']}, and fatigue level: {row['Fatigue']}. Appetite level: {row['AppetiteLevel']}, subjective health perception: {row['SubjectiveHealthPerception']}, objective health perception: {row['ObjectiveHealthPerception']}."

    # Emotions and mental state
    emotions = f"Future outlook: {row['FutureOutlook']}, fear of failure: {row['FearOfFailure']}, satisfaction level: {row['SatisfactionLevel']}. Feelings of guilt: {row['FeelingsOfGuilt']}, punishment feelings: {row['FeelingsOfPunishment']}, feelings of disappointment: {row['FeelingsOfDisappointment']}, self-blame: {row['SelfBlame']}, tendency to cry: {row['TendencyToCry']}, irritability: {row['Irritability']}, interest in others: {row['InterestInOthers']}, decision making: {row['DecisionMaking']}, cognitive retardation: {row['CognitiveRetardation']}, psychic anxiety: {row['PsychicAnxiety']}."

    # Family dynamics
    family = f"From a family of {row['SiblingsCount']} siblings, sibling order: {row['SiblingOrder']}. Parents' occupations: {row['FatherOccupation']} and {row['MotherOccupation']}, family income: {row['FamilyIncome']}, facing financial hardship: {row['FinancialHardship']}, dependent on family financially: {row['FinancialDependency']}, family illness present: {row['FamilyIllness']}. Parental communication frequency: {row['ParentalCommunicationFrequency']}, parental strictness: {row['ParentalStrictness']}, parental understanding: {row['ParentalUnderstanding']}, sibling bonding: {row['SiblingBonding']}, parental relationship stability: {row['ParentalRelationshipStability']}."

    # Social interactions
    social = f"Peer relationships described as {row['PeerRelationship']}. Work environment impact: {row['WorkEnvironmentImpact']}, societal interference: {row['SocietalInterference']}."

    # Violence experiences
    violence = f"Experiences of violence: physical {row['PhysicalViolenceExperience']}, sexual {row['SexualViolenceExperience']}."

    # Combine all aspects into a single string
    return f"{demographics} {academic} {lifestyle} {work} {health} {energy} {emotions} {family} {social} {violence}"

# Apply the function to create a new column in the DataFrame
df2 = df.apply(create_description, axis=1)


In [22]:
df2.head()

0    Female aged 23, practices Muslim, lives in Urb...
1    Female aged 24, practices Muslim, lives in Urb...
2    Male aged 24, practices Muslim, lives in Urban...
3    Male aged 22, practices Muslim, lives in Urban...
4    Male aged 25, practices Muslim, lives in Urban...
dtype: object

In [23]:
df2.to_excel('DepressionNLP.xlsx', index=False)
